In [1]:
import pandas as pd
import os
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import numpy as np

In [2]:
data = pd.read_csv("/Users/aditya/Documents/GitHub/tegger-challenge/data.csv").iloc[:,1:]
data.head()

,all,theme,title,contents
0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...
1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar..."
2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...
3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...
4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin..."


In [3]:
cv=CountVectorizer(min_df=2,stop_words='english', max_df=0.85, ngram_range=(1,3))
word_count_vector=cv.fit_transform(data['all'])

In [4]:
feature_names=cv.get_feature_names()

In [5]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=2, max_df=0.85)
tfidf = vectorizer.fit_transform(data['all'])

In [6]:
corpus = [entry.split(" ") for entry in data['all']]

In [7]:
# model = Word2Vec(corpus, min_count=2,size=10000,workers=3, window =5, sg = 0)

In [8]:
# model.save("w2v_saved.model")
model = Word2Vec.load("w2v_saved.model")

In [9]:
g20 = "Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Republic of Korea, Korea, Mexico, Russia, Saudi Arabia, Arabia, South Africa, Africa, Turkey, United Kingdom, UK, United States, America, USA, European Union, EU, Europe".lower()
g20 = g20.split(", ")
g20

['argentina',
 'australia',
 'brazil',
 'canada',
 'china',
 'france',
 'germany',
 'india',
 'indonesia',
 'italy',
 'japan',
 'republic of korea',
 'korea',
 'mexico',
 'russia',
 'saudi arabia',
 'arabia',
 'south africa',
 'africa',
 'turkey',
 'united kingdom',
 'uk',
 'united states',
 'america',
 'usa',
 'european union',
 'eu',
 'europe']

In [10]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in model.wv.vocab.keys()]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [11]:
data['mean_emb'] = data['all'].apply(lambda x: get_mean_vector(model, x.split(" ")))

In [37]:
data.head()

,id,all,theme,title,contents,mean_emb,america,argentina,australia,brazil,...,south africa,africa,turkey,united kingdom,uk,united states,usa,european union,eu,europe
0,0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...,"[7.4382326e-05, 0.046140082, 0.01195595, 0.001...",0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
1,1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar...","[0.0021663008, 0.04525567, 0.012547477, 0.0027...",0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...,"[0.004162871, 0.042615805, 0.0135024255, 0.002...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...,"[0.00523266, 0.039601512, 0.014776167, 0.00402...",1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin...","[0.0044061705, 0.03529083, 0.012877651, 0.0034...",0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [13]:
# data[['all','mean_emb']].to_csv('/Users/aditya/Documents/GitHub/tegger-challenge/mean_embeddings.csv')

In [16]:
# data = data.reset_index().rename(columns={'index':'id'})

In [22]:
# mentions = {}
# for each in g20:
#     mentions[each] = [index for index, row in data.iterrows() if "*"+each.upper() in row['all'].split(" ")]

In [31]:
# mentions['republic of korea'] = set(mentions['republic of korea']+mentions['korea'])
# mentions['saudi arabia'] = set(mentions['saudi arabia']+mentions['arabia'])
# mentions['south africa'] = set(mentions['south africa']+mentions['africa'])
# mentions['united kingdom'] = set(mentions['united kingdom']+mentions['uk'])
# mentions['united states'] = set(mentions['united states']+mentions['usa'] + mentions['america'])
# mentions['european union'] = set(mentions['european union']+mentions['eu'] + mentions['europe'])

In [36]:
for each in g20:
    data[each] = [1 if "*"+each.upper() in row['all'].split(" ") else 0 for index, row in data.iterrows()] 

In [38]:
data['republic of korea'] = data['republic of korea']+data['korea']
data['saudi arabia'] = data['saudi arabia']+data['arabia']
data['south africa'] = data['south africa']+data['africa']
data['united kingdom'] = data['united kingdom']+data['uk']
data['united states'] = data['united states']+data['usa'] + data['america']
data['european union'] = data['european union']+data['eu'] + data['europe']

In [40]:
data.drop(columns=['korea', 'arabia', 'africa', 'uk', 'america' , 'usa', 'eu', 'europe'], inplace=True)